Consider batch with a single feature. Let $x = \in \mathbb{R}^B$ be input batch, $\hat{x} \in \mathbb{R}^B$ - output batch. Define:
\begin{gather*}
    t_i = x_i - \frac{1}{B}\sum_{j=1}^B x_j\\
    s = \sqrt{\sigma^2 + \varepsilon}
\end{gather*}
I.e.:
\begin{gather*}
    \hat{x}_i = t_i / s \\
    \sigma^2 = \frac{1}{B}\sum t_i
\end{gather*}
Notice, that:
$$
    \frac{d s}{d t_j} = \frac{t_j}{Bs}
$$
Therefore:
\begin{align*}
    \frac{d\hat{x_i}}{d t_j}
    &= \frac{\delta_{ij}}{s} - \frac{t_i}{s^2} \frac{d s}{d t_j} =\\
    &= \frac{\delta_{ij}}{s} - \frac{t_i t_j}{Bs^3}
\end{align*}
So, respective Jacobian matrix equals:
$$
    J_{t, \hat{x}} = \frac{1}{s}I_B - \frac{1}{Bs^3}t t^T
$$
Similarly for $t \to x$:
$$
    \frac{d t_i}{d x_j} = \delta_{ij} - \frac{1}{B}
$$
$$
    J_{x, t} = I_B - \frac{1}{B} 1_B 1_B^T
$$
Thanks to simple form of Jacobian matrices, we can use just operations of O(B) complexity while calculating input gradient:
$$
    \frac{dL}{dx} = J_{x, t}J_{t, \hat{x} }\frac{dL}{d\hat{x}}
$$
Multiplication by $1_B^T$ is sum along batch dimension. Multiplication by $1_B^T$ is broadcasting along batch dimension. Multiplication $t^T v$ is coordinate-wise multiplication plus sum along batch dimension. Therefore with naive algorithm we would need to synchronize batches twice.

However if we denote $g := \frac{dL}{d\hat{x}}$ and elaborate on the final formula, we get:
\begin{align*}
    \frac{dL}{dx} &= \Big(I_B - \frac{1}{B} 1_B 1_B^T\Big)\Big(g/s - \frac{1}{Bs^3}t t^T g\Big)\\
    &= g/s - \frac{1}{Bs^3}t t^T g - \frac{1}{Bs} 1_B 1_B^Tg + \frac{1}{B^2s^3}1_B 1_B^T t t^T g\\
\end{align*}
Therefore, we need to calculate only three sums over **B**atch dimension: $t * g$, $1 * g$, $1 * t$. All three sums can be calculated with just 1 synchronize.

Notice, that for multi-feature inputs result can be simply broadcasted on feature dimension.